В данной лабораторнйо работе выполнено перемножение матриц с помощью CPU, CPU с подключенным распараллеливанием. Также одним из способов является использование библиотеки Numba, позволяющая использовать CUDA для распараллеливания на GPU. 

Для CUDA мы использовали два варианта: 

1. Через глобальную память;
2. Через общую память.

P.s: общая память повзоляет производить вычисления намного быстрее.

In [3]:
# добавляем используемые библиотеки
import math
import time
import numpy as np
from numba import cuda, jit, float64
import matplotlib.pyplot as plt
from tabulate import tabulate


 #1024, 1500, 2024, 4048, 8096
for k in [128, 256, 512, 1024, 1500, 2024, 4048]:
  #for l in [128, 256, 512, 1024]:
    TPB = 16 # thread per block - Определяем количество потоков в блоке 

    #N = l   
                   # 128/256/512/1024
    A = np.random.random((k, k))   #Матрицы для умножения
    B = np.random.random((k, k))
    C = np.zeros((k, k))      # результат перемножения матриц

    #Вычисление перемножения на CPU силами самого CPU

    def cpu_mat_mul(A, B, C):
            C = np.dot(A, B)
        
    start = time.time() # начало отсчёта времени выполнения программы 
    cpu_mat_mul(A, B, C) # функция перемножения матриц средствами только CPU
    cpu = C            #записываем резулттат перемножения в спец.переменную
    #print('Перемножение на CPU:', time.time()-start)

    # Аннтотация @jit - означет что код должен быть скомпилировать специальным образом для многопоточных вычислений на CPU
    @jit
    def cpu_mat_mul_jit(A, B, C):   # матрица перемноженная только силами CPU с многопоточностью
        C = np.dot(A, B)

    start = time.time()
    cpu_mat_mul_jit(A, B, C)
    cpu_git = C
    #print('Перемножение на CPU с подключением многопоточности:', time.time()-start)

    # Аннтотация @cuda.jit - означет что код должен быть скомпилировать специальным образом для многопоточных вычислений на GPU

    # В данной функции используется наивный способ вычислений с помощью CUDA на GPU для распараллеливания вычислений
    @cuda.jit
    def mat_mul_naive_kernal(A, B, C):
        
        #Определяем текущую позицию двумерной сетки в нашей видеокарты
        #Это требуется для корректного выделения блоков с потоками вычисления
        i, j = cuda.grid(2)
        if i < C.shape[0] and j < C.shape[1]: #перемножение матриц
            summation = 0
            for k in range(A.shape[1]):
                summation += A[i, k] * B[k, j]
            C[i, j] = summation

    def host_naive(A, B, C):
        '''host code for calling naive kernal
        '''
        d_A = cuda.to_device(A)  # отправляем наши массивы на девайс (GPU) для быстрого взаимодействия с ними
        d_B = cuda.to_device(B)
        d_C = cuda.device_array(C.shape, np.float64) #выделяем место под наш массив с размером, который задали для массива C

        threadsperblock = (TPB, TPB) # собственно сам блок на "кол-во нитей" на  "кол-во нитей"
        blockspergrid_x = math.ceil(A.shape[0]/threadsperblock[0])
        blockspergrid_y = math.ceil(B.shape[1]/threadsperblock[1])
        blockspergrid = (blockspergrid_x, blockspergrid_y) # 

        mat_mul_naive_kernal[blockspergrid, threadsperblock](d_A, d_B, d_C) #Аргументы в квадратных скобках определяют размер 
                                                                    #и форму сетки потока, а аргументы в круглых скобках соответствуют аргументам функции ядра 

        return d_C.copy_to_host()

    start = time.time()
    host_naive(A, B, C)
    c_u_d_a_naive = C
    #print('Перемножение на GPU наивным способом:', time.time()-start)

    # В данной функции используется наивный способ вычислений с помощью CUDA на GPU для распараллеливания вычислений
    # но с использованием общих объёмов памяти, что позволяет повысить производительность

    @cuda.jit
    def mat_mul_shared_kernal(A, B, C):
        
        #Выделяем память на видеокарте

        # Определяем массив в общей памяти
        # Размер и тип массивов должны быть известны во время компиляции
        s_A = cuda.shared.array((TPB, TPB), dtype=float64)  # выделяем на ядре GPU память для будущего массива 
        s_B = cuda.shared.array((TPB, TPB), dtype=float64)
        
        x, y = cuda.grid(2) #Определяем текущую позицию двумерной сетки в нашей видеокарты
        tx = cuda.threadIdx.x  #это уникальный идентификатор потока в одномерном блоке
        ty = cuda.threadIdx.y
        bw = cuda.blockDim.x #Аналогично, это уникальный идентификатор блока в 1D-сетке
        bh = cuda.blockDim.y 
        #print((x, y), (tx, ty), (bx, by), (bw, bh))

        if x >= C.shape[0] or y >= C.shape[1]:
            return

        tmp = 0
        for i in range(int(A.shape[1]/TPB)):
            #print((x, y), (tx, ty), i)
            s_A[tx, ty] = A[x, ty + bw*i] # записываем в каждый элемент массива на GPU элементы массива
            s_B[tx, ty] = B[tx + bh*i, y]

            #Требуется чтоб все потоки завершелись одновременно, т.к. изначально они все ассинхронны 
            cuda.syncthreads()

            for j in range(TPB):
                tmp += s_A[tx, j] * s_B[j, ty]

            cuda.syncthreads()
        C[x, y] = tmp # собственно наш результат

    def host_optimized(A, B, C):
        '''host code for calling naive kernal
        '''
        d_A = cuda.to_device(A)  # d_ --> device
        d_B = cuda.to_device(B)
        d_C = cuda.device_array(C.shape, np.float64)

        threadsperblock = (TPB, TPB)
        blockspergrid_x = math.ceil(A.shape[0]/threadsperblock[0])
        blockspergrid_y = math.ceil(B.shape[1]/threadsperblock[1])
        blockspergrid = (blockspergrid_x, blockspergrid_y)

        mat_mul_shared_kernal[blockspergrid, threadsperblock](d_A, d_B, d_C) #Аргументы в квадратных скобках определяют размер 
                                                                    #и форму сетки потока, а аргументы в круглых скобках соответствуют аргументам функции ядра
        return d_C.copy_to_host()

    start = time.time()
    host_optimized(A, B, C)
    c_u_d_a_shared = C
    #print('Перемножение на GPU с общей памятью:', time.time()-start)

    def main():
        #rows = []
        print('Матрица', k, 'на', k)
        start = time.time()
        lol_1 = cpu_mat_mul(A, B, C)
        kek_1 = time.time()-start
        
        print('Время выполнения вычислений на CPU:', kek_1, end='\n')
          #plt.plot(k, kek_1)
          #print(lol_graph_1)
          

        start = time.time()
        lol_2 = cpu_mat_mul_jit(A, B, C)
        kek_2 = time.time()-start
        print('Время выполнения вычислений на CPU Numba.jit:', kek_2, end='\n')

        start = time.time()
        lol_3 = host_naive(A, B, C)
        kek_3 = time.time()-start
        print('Время выполнения вычислений на GPU global:', kek_3, end='\n')
          #print(ans)
          
        start = time.time()
        lol_4 = host_optimized(A, B, C)
        kek_4 = time.time()-start
        print('Время выполнения вычислений на GPU shared:', kek_4, end='\n')
        print('Разница с лучшим результатом распараллеливания:', kek_1 - min(kek_2, kek_3, kek_4))
        print("-" * 80)
          #print(ans)
          #kek = [kek_1, kek_2, kek_3, kek_4]
        #row = [k, kek_1, kek_2, kek_3, kek_4]
        #rows.append(row)
    #print(tabulate(rows, headers=['matrix size', 'CPU', 'CPU Numba.jit', 'GPU global', 'GPU shared']))
       

    if __name__ == '__main__':
      main()
    

    #print('Простое CPU и GPU shared::', np.allclose(cpu, c_u_d_a_shared), end='\n\n')

    #print('Простое CPU с распараллеливанием и GPU shared::', np.allclose(cpu_git, c_u_d_a_shared), end='\n\n')

    #print("Данные представлены с использованием:", TPB,"потоков в блоке и", "размерностью массивов", k, "на", k, end='\n\n\n\n\n\n')
  
    

Матрица 128 на 128
Время выполнения вычислений на CPU: 0.004070758819580078
Время выполнения вычислений на CPU Numba.jit: 0.0002231597900390625
Время выполнения вычислений на GPU global: 0.003416776657104492
Время выполнения вычислений на GPU shared: 0.0028014183044433594
Разница с лучшим результатом распараллеливания: 0.0038475990295410156
--------------------------------------------------------------------------------
Матрица 256 на 256
Время выполнения вычислений на CPU: 0.004828214645385742
Время выполнения вычислений на CPU Numba.jit: 0.0013797283172607422
Время выполнения вычислений на GPU global: 0.006148338317871094
Время выполнения вычислений на GPU shared: 0.005535602569580078
Разница с лучшим результатом распараллеливания: 0.003448486328125
--------------------------------------------------------------------------------
Матрица 512 на 512
Время выполнения вычислений на CPU: 0.011809110641479492
Время выполнения вычислений на CPU Numba.jit: 0.012165307998657227
Время выполнен